In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd

os.chdir("/content/drive/MyDrive/교통속도_origin_data")

In [4]:
id_list = [1040012100,1040012000,1040011100, 1040011000,1040010100,1040010000,1040009100,1040009000]

In [8]:
import pandas as pd
df = pd.DataFrame()

for i in range(1,13):
    df = df.append(pd.read_excel('2022년 '+str(i)+'월 서울시 차량통행속도.xlsx'))

<ipython-input-8-7b0b6d8b9c7f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_excel('2022년 '+str(i)+'월 서울시 차량통행속도.xlsx'))
<ipython-input-8-7b0b6d8b9c7f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_excel('2022년 '+str(i)+'월 서울시 차량통행속도.xlsx'))
<ipython-input-8-7b0b6d8b9c7f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_excel('2022년 '+str(i)+'월 서울시 차량통행속도.xlsx'))
<ipython-input-8-7b0b6d8b9c7f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_excel('2022년 '+str(i)+'월 서울시 차량통행속도.xlsx'))
<ipython-input-8-7b0b6d8b9c7f>:5: FutureWarning: The fra

In [9]:
df = df.loc[df['링크아이디'].isin(id_list)]

In [10]:
df.to_csv("final_통행속도_2022.csv", encoding = "cp949")

In [13]:
for i in id_list:
    a = df[df['링크아이디']==i]
    a = a.set_index("일자").drop(['요일','도로명','기능유형구분','차선수','거리','도심/외곽구분','권역구분','시점명','종점명'], axis = 1)
    a = a.drop([], axis =1)
    a = a.reset_index()
    a = pd.melt(a, id_vars=['링크아이디','방향','일자'])
    a['time'] = a['일자'].map(str) + "" + a.variable.str.rstrip("시")
    a = a.set_index(["time"]).drop(['일자','variable'], axis = 1)
    a.to_csv(str(i)+"통행속도_2022.csv", encoding = "cp949")

In [14]:
df = pd.read_csv("통행량_2022.csv", encoding = "cp949")
df['time'] = pd.to_datetime(df['time'])

In [15]:
# 24시를 00시로 바꾸기
from datetime import datetime, timedelta

def zerofrom24(datetime, _format):
    datetime = str(datetime)
    try:
        return pd.to_datetime(datetime, format = _format)
    except:
        datetime = datetime[:-2] + '00'
        return pd.to_datetime(datetime, format = _format)+ timedelta(days=1)


In [16]:
for i in id_list:
  s = pd.read_csv(str(i)+"통행속도_2022.csv", encoding = "cp949")
  s.time = s.time.map(lambda x : zerofrom24(x,'%Y%m%d%H'))
  s = s.set_index(["time"])
  mm = pd.merge(df, s, on = "time").drop(["링크아이디", "방향"], axis = 1)
  no_mm = mm.dropna(axis = 0) # 결측치 있는 행 전부 삭제
  no_mm.to_csv("final_"+str(i)+".csv", encoding = "cp949")